# Scratchwork

This notebook is only used for trying out ideas.

In [ ]:
import algebras as alg
import json
import os
import numpy as np
import itertools as it
from pprint import pprint

In [ ]:
# Path to this repo
aa_path = os.path.join(os.getenv('PYPROJ'), 'abstract_algebra')

# Path to a directory containing Algebra definitions in JSON
alg_dir = os.path.join(aa_path, "Algebras")

## Groups for Testing

### D4 - Dihedral Group on 4 Vertices

In [ ]:
d4_path = os.path.join(alg_dir, "d4_dihedral_group_on_4_vertices.json")
!cat {d4_path}

In [ ]:
d4 = alg.Group(d4_path)
d4.print_info()

### V4 - Klein 4 Group

In [ ]:
v4_path = os.path.join(alg_dir, "v4_klein_4_group.json")
!cat {v4_path}

In [ ]:
v4 = alg.Group(v4_path)
v4.print_info()

## Changing the Order of Elements

<b>Observation</b>: If the group is abelian, then changing the order of the elements does <b>not</b> affect the order of the indices in the <i>mult_table</i>.  <b>Why?</b>

In [ ]:
# new_elem_order = v4.element_names
# new_v4_elem_order = ['e', 'v', 'h', 'hv']
new_v4_elem_order = ["e", "hv", "h", "v"]

In [ ]:
#table_from_reordered_elements(v4, new_v4_elem_order)
v4_reordered = v4.reorder_elements(new_v4_elem_order)
v4_reordered

In [ ]:
pprint(v4_reordered.mult_table_with_names())

In [ ]:
v4_reordered.mult_table

In [ ]:
pprint(v4.mult_table_with_names())

In [ ]:
v4.mult_table

In [ ]:
new_d4_elem_order = ["e", "r", "r^2", "f", "r^3", "fr", "r^2f", "rf"]

In [ ]:
d4_reordered = d4.reorder_elements(new_d4_elem_order)
d4_reordered

In [ ]:
d4_reordered.mult_table

In [ ]:
d4.mult_table

In [ ]:
d4.element_orders()

In [ ]:
d4_reordered.element_orders()

## Generating Possible Multiplication Tables

In [1]:
from itertools import product, permutations, combinations

def no_conflict(p1, p2):
    """Returns True only if no element of p1 equals the corresponding element of p2."""
    return all([p1[i] != p2[i] for i in range(len(p1))])

def no_conflicts(items):
    """Return True if each possible pair, from a list of items, has no conflicts."""
    return all(no_conflict(combo[0], combo[1]) for combo in combinations(items, 2))

def filter_out_conflicts(perms, perm, n):
    """Filter out all permutations in perms that confict with perm,
    and don't have n as the first element."""
    nperms = [q for q in perms if q[0] == n]
    return [p for p in nperms if no_conflict(p, perm)]

def generate_all_group_tables(order):
    """Return a list of all arrays that correspond to multiplication tables for groups of a specific order """
    row0 = list(range(order))
    all_perms = permutations(row0)
    row_candidates = [[row0]]
    for row_num in range(1, order):
        row_candidates.append(filter_out_conflicts(permutations(row0), row0, row_num))
    table_candidates = list(product(*row_candidates))
    return [tbl for tbl in table_candidates if no_conflicts(tbl)]

In [2]:
foo = generate_all_group_tables(4)
foo

[([0, 1, 2, 3], (1, 0, 3, 2), (2, 3, 0, 1), (3, 2, 1, 0)),
 ([0, 1, 2, 3], (1, 0, 3, 2), (2, 3, 1, 0), (3, 2, 0, 1)),
 ([0, 1, 2, 3], (1, 2, 3, 0), (2, 3, 0, 1), (3, 0, 1, 2)),
 ([0, 1, 2, 3], (1, 3, 0, 2), (2, 0, 3, 1), (3, 2, 1, 0))]

In [3]:
order = 4

In [ ]:
row0 = list(range(order))
row0

In [ ]:
# all_perms = list(map(lambda perm: list(perm), permutations(row0)))
all_perms = permutations(row0)
all_perms

In [ ]:
rows1a = filter(permutations(row0), row0, 1)
rows1a

In [ ]:
rows2a = filter(permutations(row0), row0, 2)
rows2a

In [ ]:
rows3a = filter(permutations(row0), row0, 3)
rows3a

In [ ]:
x = [[row0], rows1a, rows2a, rows3a]
x

In [ ]:
ts = list(product(*x))

In [ ]:
foo = ts[0]
foo

In [ ]:
for combo in combinations(foo, 2):
    print(combo, no_conflict(combo[0], combo[1]))

In [ ]:
def no_conflicts(items):
    return all(no_conflict(combo[0], combo[1]) for combo in combinations(items, 2))

In [ ]:
no_conflicts(foo)

In [ ]:
for t in ts:
    if no_conflicts(t):
        print(t)

In [ ]:
[t for t in ts if no_conflicts(t)]